<a href="https://colab.research.google.com/github/argonism/TsukurinagaraRL/blob/master/Zerokara_chap6_DDQN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DDQN(Double-DQN)

Fixed Target Q-Networkを、ちゃんと実装したやつ

更新式も変わる。
$$Q_m(s_t, a_t) = Q_m(s_t, a_t) + \eta * (R_{t+1} + \gamma \max_aQ_t(s_{t+1}, a_t) - Q_m(s_t, a_t))$$
結構変わってない？？？


ちょっと試した程度では効果は実感できなかった。



In [ ]:
# 使用するパッケージのインストール
# gym==0.17.2 pyvirtualdisplay==1.3.2
# xvfb=2:1.19.6-1ubuntu4.4 python-opengl=3.1.0+dfsg-1 ffmpeg=7:3.4.8-0ubuntu0.2
# JSAnimation==0.1
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1
!pip install JSAnimation > /dev/null 2>&1

In [ ]:
 import numpy as np
 import matplotlib.pyplot as plt
 %matplotlib inline
 import gym
from gym.wrappers import Monitor

In [ ]:
# 動画の描画関数の宣言
import glob
import io
import os
import base64
from JSAnimation.IPython_display import display_animation
from matplotlib import animation
from IPython import display as ipythondisplay
from IPython.display import HTML
from pyvirtualdisplay import Display

display = Display(visible=0, size=(640, 400))
display.start()

def show_video():
  mp4list = glob.glob('video/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")
    
def reset_video():
  mp4list = glob.glob('video/*.mp4')
  for mp4 in mp4list:
    os.remove(mp4)

def wrap_env(env):
  env = Monitor(env, './video', force=True, video_callable=(lambda ep: ep % 10 == 0))
  reset_video()
  return env

In [ ]:
from collections import namedtuple
Tr = namedtuple('tr', ('name_a', 'value_b'))
Tr_object = Tr('名前Aです', 100)
print(Tr_object)
print(Tr_object.value_b)
Transition = namedtuple('Transition', ('state', 'action', 'next_state', 'reward'))


ENV = 'CartPole-v0'
GAMMA = 0.99
MAX_STEPS = 200
NUM_EPISODES = 500

tr(name_a='名前Aです', value_b=100)
100


In [ ]:
class ReplayMemory:
  def __init__(self, CAPACITY):
    self.capacity = CAPACITY
    self.memory = []
    self.index = 0
  
  def push(self, state, action, state_next, reward):
    if len(self.memory) < self.capacity:
      self.memory.append(None)
    
    self.memory[self.index] = Transition(state, action, state_next, reward)

    self.index = (self.index + 1) % self.capacity
  
  def sample(self, batch_size):
    return random.sample(self.memory, batch_size)
  
  def __len__(self):
    return len(self.memory)


In [ ]:
from torch import nn
import torch.nn.functional as F

class Net(nn.Module):

  def __init__(self, n_in, n_mid, n_out):
    super(Net, self).__init__()
    self.fc1 = nn.Linear(n_in, n_mid)
    self.fc2 = nn.Linear(n_mid, n_mid)
    self.fc3 = nn.Linear(n_mid, n_out)
  
  def forward(self, x):
    h1 = F.relu(self.fc1(x))
    h2 = F.relu(self.fc2(h1))
    output = self.fc3(h2)
    return output

In [ ]:
import random
import torch
from torch import nn
import torch.nn.functional as F
from torch import optim

BATCH_SIZE = 32
CAPACITY = 10000

class Brain:
  def __init__(self, num_states, num_actions):
    self.num_actions = num_actions

    self.memory = ReplayMemory(CAPACITY)

    n_in, n_mid, n_out = num_states, 32, num_actions
    self.main_q_network = Net(n_in, n_mid, n_out)
    self.target_q_network = Net(n_in, n_mid, n_out)
    print(self.main_q_network)

    self.optimizer = optim.Adam(self.main_q_network.parameters(), lr=0.0001)
  
  def replay(self):
    '''保存した行動や結果、状態から結合パラメータを学習する'''

    if len(self.memory) < BATCH_SIZE:
      return
    
    # ミニバッチの作成
    self.batch, self.state_batch, self.action_batch, self.reward_batch, self.non_final_next_states = self.make_minibatch()

    # 教師信号となるQ(s_t, a_t)を求める
    self.expected_state_action_values = self.get_expected_state_action_values()

    # 訓練
    self.update_main_q_network()
  
  def update_main_q_network(self):
    self.main_q_network.train()

    # smooth_l1_lossはHuber関数
    # expected_state_action_valuesはsize: バッチサイズとなっているので、これをunsqueezeで(バッチサイズ*1)にする。...?
    loss = F.smooth_l1_loss(self.state_action_values, self.expected_state_action_values.unsqueeze (1))
    
    self.optimizer.zero_grad()
    loss.backward()
    self.optimizer.step()
  
  def make_minibatch(self):
      '''2. ミニバッチの作成'''

      # 2.1 メモリからミニバッチ分のデータを取り出す
      transitions = self.memory.sample(BATCH_SIZE)

      # 2.2 各変数をミニバッチに対応する形に変形
      # transitionsは1stepごとの(state, action, state_next, reward)が、BATCH_SIZE分格納されている
      # つまり、(state, action, state_next, reward)×BATCH_SIZE
      # これをミニバッチにしたい。つまり
      # (state×BATCH_SIZE, action×BATCH_SIZE, state_next×BATCH_SIZE, reward×BATCH_SIZE)にする
      batch = Transition(*zip(*transitions))

      # 2.3 各変数の要素をミニバッチに対応する形に変形し、ネットワークで扱えるようVariableにする
      # 例えばstateの場合、[torch.FloatTensor of size 1x4]がBATCH_SIZE分並んでいるのですが、
      # それを torch.FloatTensor of size BATCH_SIZEx4 に変換します
      # 状態、行動、報酬、non_finalの状態のミニバッチのVariableを作成
      # catはConcatenates（結合）のことです。
      state_batch = torch.cat(batch.state)
      action_batch = torch.cat(batch.action)
      reward_batch = torch.cat(batch.reward)
      non_final_next_states = torch.cat([s for s in batch.next_state
                                          if s is not None])

      return batch, state_batch, action_batch, reward_batch, non_final_next_states

  def get_expected_state_action_values(self):
    self.main_q_network.eval()
    self.target_q_network.eval()
    # self.model(state_batch) でbatch_size*2のテンソル(actionのQ値)が出る。
    # action_batchはbatch_size*(0, 1)のようになっている。
    # その後実行したactionの方のQ値を取り出すために、gatherを使っている。
    self.state_action_values = self.main_q_network(self.state_batch).gather(1, self.action_batch)

    # 次の状態(next_state)があるかどうかに注意しながら取り出し、max{Q(s_t+1, a)}を求める
    # ここではインデックスマスクを作る。...?
    # インデックスにByteTensor入れると、ByteTensorでTrueになってる要素を取り出して、そのそれぞれに対して代入を行えるっぽい。
    non_final_mask = torch.ByteTensor(tuple(map(lambda s: s is not None, self.batch.next_state)))

    next_state_values = torch.zeros(BATCH_SIZE)

    a_m = torch.zeros(BATCH_SIZE).type(torch.LongTensor)

    a_m[non_final_mask] = self.main_q_network(self.non_final_next_states).detach().max(1)[1]

    a_m_non_final_next_states = a_m[non_final_mask].view(-1, 1)

    # 次の状態をモデルに入力して、次の行動のQ値が大きいものを取り出す。
    # それをnext_state_valuesの対応するインデックスに代入していく。
    # 後々微分されないために（固定するために）detach()をする。くわしくはわからん。
    # 現在の状態についてのQ値は更新対象であるから、detachしないっぽい。
    next_state_values[non_final_mask] = self.target_q_network(self.non_final_next_states).gather(1, a_m_non_final_next_states).detach().squeeze()

    # 教師となるQ値を求める。説明で書いた式の右辺。
    expected_state_action_values = self.reward_batch + GAMMA * next_state_values

    return expected_state_action_values
  
  def update_target_q_network(self):
    self.target_q_network.load_state_dict(self.main_q_network.state_dict())

  def decide_action(self, state, episode):
    epsilon = 0.5 * (1 / (episode + 1))

    if epsilon <= np.random.uniform(0, 1):
      self.main_q_network.eval()
      with torch.no_grad():
        action = self.main_q_network(state).max(1)[1].view(1, 1)
        # view(1, 1)は、[torch.LongTensor of size 1]をsize 1x1にするらしい。逆にsize 1x1って何。

    else:
      action = torch.LongTensor([[random.randrange(self.num_actions)]])
    
    return action



In [ ]:
t = torch.zeros(10)
mask = t is not None
new = torch.tensor([[1, 0, 1, 1, 0, 1, 0, 0, 1, 1]])
print(t[mask])
t[mask] = new
print(t)

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])
tensor([1., 0., 1., 1., 0., 1., 0., 0., 1., 1.])


In [ ]:
class Agent:
  def __init__(self, num_states, num_actions):
    self.brain = Brain(num_states, num_actions)
  
  def update_q_function(self):
    self.brain.replay()
  
  def get_action(self, state, episode):
    action = self.brain.decide_action(state, episode)
    return action

  def memorize(self, state, action, state_next, reward):
    self.brain.memory.push(state, action, state_next, reward)

  def update_target_q_function(self):
    self.brain.update_target_q_network()
  

In [ ]:
class Environment:
  def __init__(self):
    self.env = wrap_env(gym.make(ENV))
    self.num_states = self.env.observation_space.shape[0]
    self.num_actions = self.env.action_space.n

    self.agent = Agent(self.num_states, self.num_actions)
  
  def run(self):
    # 10回ごとに立ち続けたepisodeの平均を取る
    episode_10_list = np.zeros(10)
    # 連続成功記録
    complete_episodes = 0
    episode_final = False

    for episode in range(NUM_EPISODES):
      observation = self.env.reset()

      state = observation
      state = torch.from_numpy(state).type(torch.FloatTensor)
      state = torch.unsqueeze(state, 0)

      for step in range(MAX_STEPS):
        
        action = self.agent.get_action(state, episode)

        observation_next, _, done, _ = self.env.step(action.item())

        if done:
          state_next = None
          episode_10_list = np.hstack((episode_10_list[1:], step + 1))

          if step < 195:
            reward = torch.FloatTensor([-1.0])
            complete_episodes = 0
          else:
            reward = torch.FloatTensor([1.0])
            complete_episodes += 1
        else:
          reward = torch.FloatTensor([0.0])
          state_next = observation_next
          state_next = torch.from_numpy(state_next).type(torch.FloatTensor)
          state_next = torch.unsqueeze(state_next, 0)
        
        self.agent.memorize(state, action, state_next, reward)

        self.agent.update_q_function()

        state = state_next

        if done:
          print(f'{episode} Episode: Finished after {step + 1} time steps: 10思考の平均step数 = {episode_10_list.mean()}')

          if episode % 2 == 0:
            self.agent.update_target_q_function()
          break
        
      if episode_final is True:
        show_video()
        break
      
      if complete_episodes >= 10:
        print('10回連続成功')
        episode_final = True


In [ ]:
cartpole_env = Environment()
cartpole_env.run()

Net(
  (fc1): Linear(in_features=4, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=2, bias=True)
)
0 Episode: Finished after 17 time steps: 10思考の平均step数 = 1.7
1 Episode: Finished after 17 time steps: 10思考の平均step数 = 3.4
2 Episode: Finished after 13 time steps: 10思考の平均step数 = 4.7
3 Episode: Finished after 15 time steps: 10思考の平均step数 = 6.2
4 Episode: Finished after 20 time steps: 10思考の平均step数 = 8.2
5 Episode: Finished after 19 time steps: 10思考の平均step数 = 10.1


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:92: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at  /pytorch/aten/src/ATen/native/IndexingUtils.h:25.)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:94: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at  /pytorch/aten/src/ATen/native/IndexingUtils.h:25.)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:100: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at  /pytorch/aten/src/ATen/native/IndexingUtils.h:25.)


6 Episode: Finished after 26 time steps: 10思考の平均step数 = 12.7
7 Episode: Finished after 15 time steps: 10思考の平均step数 = 14.2
8 Episode: Finished after 17 time steps: 10思考の平均step数 = 15.9
9 Episode: Finished after 23 time steps: 10思考の平均step数 = 18.2


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:92: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at  /pytorch/aten/src/ATen/native/IndexingUtils.h:25.)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:94: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at  /pytorch/aten/src/ATen/native/IndexingUtils.h:25.)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:100: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at  /pytorch/aten/src/ATen/native/IndexingUtils.h:25.)


10 Episode: Finished after 17 time steps: 10思考の平均step数 = 18.2
11 Episode: Finished after 19 time steps: 10思考の平均step数 = 18.4
12 Episode: Finished after 41 time steps: 10思考の平均step数 = 21.2
13 Episode: Finished after 31 time steps: 10思考の平均step数 = 22.8
14 Episode: Finished after 19 time steps: 10思考の平均step数 = 22.7
15 Episode: Finished after 62 time steps: 10思考の平均step数 = 27.0
16 Episode: Finished after 25 time steps: 10思考の平均step数 = 26.9
17 Episode: Finished after 27 time steps: 10思考の平均step数 = 28.1
18 Episode: Finished after 17 time steps: 10思考の平均step数 = 28.1
19 Episode: Finished after 19 time steps: 10思考の平均step数 = 27.7


...

240 Episode: Finished after 200 time steps: 10思考の平均step数 = 183.2
241 Episode: Finished after 117 time steps: 10思考の平均step数 = 182.0
242 Episode: Finished after 200 time steps: 10思考の平均step数 = 182.0
243 Episode: Finished after 200 time steps: 10思考の平均step数 = 182.0
244 Episode: Finished after 200 time steps: 10思考の平均step数 = 182.0
245 Episode: Finished after 200 time steps: 10思考の平均step数 = 187.9
246 Episode: Finished after 200 time steps: 10思考の平均step数 = 191.7
247 Episode: Finished after 200 time steps: 10思考の平均step数 = 191.7
248 Episode: Finished after 122 time steps: 10思考の平均step数 = 183.9
249 Episode: Finished after 143 time steps: 10思考の平均step数 = 178.2


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:92: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at  /pytorch/aten/src/ATen/native/IndexingUtils.h:25.)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:94: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at  /pytorch/aten/src/ATen/native/IndexingUtils.h:25.)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:100: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at  /pytorch/aten/src/ATen/native/IndexingUtils.h:25.)


250 Episode: Finished after 194 time steps: 10思考の平均step数 = 177.6
251 Episode: Finished after 200 time steps: 10思考の平均step数 = 185.9
252 Episode: Finished after 200 time steps: 10思考の平均step数 = 185.9
253 Episode: Finished after 191 time steps: 10思考の平均step数 = 185.0
254 Episode: Finished after 200 time steps: 10思考の平均step数 = 185.0
255 Episode: Finished after 200 time steps: 10思考の平均step数 = 185.0
256 Episode: Finished after 191 time steps: 10思考の平均step数 = 184.1
257 Episode: Finished after 200 time steps: 10思考の平均step数 = 184.1
258 Episode: Finished after 191 time steps: 10思考の平均step数 = 191.0
259 Episode: Finished after 200 time steps: 10思考の平均step数 = 196.7


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:92: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at  /pytorch/aten/src/ATen/native/IndexingUtils.h:25.)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:94: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at  /pytorch/aten/src/ATen/native/IndexingUtils.h:25.)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:100: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at  /pytorch/aten/src/ATen/native/IndexingUtils.h:25.)


260 Episode: Finished after 200 time steps: 10思考の平均step数 = 197.3
261 Episode: Finished after 200 time steps: 10思考の平均step数 = 197.3
262 Episode: Finished after 200 time steps: 10思考の平均step数 = 197.3
263 Episode: Finished after 200 time steps: 10思考の平均step数 = 198.2
264 Episode: Finished after 200 time steps: 10思考の平均step数 = 198.2
265 Episode: Finished after 200 time steps: 10思考の平均step数 = 198.2
266 Episode: Finished after 200 time steps: 10思考の平均step数 = 199.1
267 Episode: Finished after 200 time steps: 10思考の平均step数 = 199.1
268 Episode: Finished after 200 time steps: 10思考の平均step数 = 200.0
10回連続成功
269 Episode: Finished after 186 time steps: 10思考の平均step数 = 198.6
